In [49]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from DeepPurpose.pybiomed_helper import _GetPseudoAAC, CalculateAADipeptideComposition, \
calcPubChemFingerAll, CalculateConjointTriad, GetQuasiSequenceOrder
import torch
from torch.utils import data
from torch.autograd import Variable
try:
	from descriptastorus.descriptors import rdDescriptors, rdNormalizedDescriptors
except:
	raise ImportError("Please install pip install git+https://github.com/bp-kelley/descriptastorus.")
from DeepPurpose.chemutils import get_mol, atom_features, bond_features, MAX_NB, ATOM_FDIM, BOND_FDIM
from subword_nmt.apply_bpe import BPE
import codecs
import pickle
import wget
from zipfile import ZipFile 
import os
import sys

from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import SequentialSampler
from torch import nn 

from tqdm import tqdm
import matplotlib.pyplot as plt
from time import time
from sklearn.metrics import mean_squared_error, roc_auc_score, average_precision_score, f1_score, log_loss
from lifelines.utils import concordance_index
from scipy.stats import pearsonr
import pickle 
torch.manual_seed(2)
np.random.seed(3)
import copy
from prettytable import PrettyTable

import os

from DeepPurpose.utils import *
from DeepPurpose.model_helper import Encoder_MultipleLayers, Embeddings        
from DeepPurpose.encoders import *
from DeepPurpose import DTI

import collections

In [84]:
df = pd.read_pickle("./df.pkl")

idx_str = ['Kd', 'IC50', 'Ki','EC50','Temp']
df_want = df
convert_to_log = 0

# have at least uniprot or pubchem ID
df_want = df_want[df_want.PubChem_ID.notnull() | df_want.UniProt_ID.notnull()]
df_want = df_want[df_want.InChI.notnull()]

for label in idx_str:
    df_want[label] = df_want[label].str.replace('>', '')
    df_want[label] = df_want[label].str.replace('<', '')

y = df_want[idx_str]
for label in idx_str:
    if convert_to_log:
            print('Default set to logspace (nM -> p) for easier regression')
            y[label] = convert_y_unit(df_want[label].values, 'nM', 'p') 
    else:
            y[label] = df_want[label].values

y['pH'] = df_want['pH']
X_drugs = df_want.SMILES.values
X_targets = df_want['Target Sequence'].values
y = y.apply(pd.to_numeric, errors='coerce')
X_org = df_want['Organism']

C:\ProgramData\Anaconda3\envs\multiPurpose\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\multiPurpose\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
y

In [85]:
df_backup = df_data
df_data = df_data.sample(10000)

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer

imputer = SimpleImputer() #TODO: better imputer
y_i = imputer.fit_transform(y)
y_i = pd.DataFrame(data=y_i,columns=idx_str+['pH'])

df_data = y_i
df_data['SMILES'] = X_drugs
df_data['Target Sequence'] = X_targets
df_data['Organism'] = X_org
df_data = df_data.join(y, rsuffix='_raw')
df_data.head()

In [52]:
df_data = df_backup

In [54]:
import math
def magnitude(x):
    if x > 0:
        return int(math.floor(math.log10(x)))
    else:
        return 0

df_data['magic50'] = df_data['IC50'].apply(magnitude)

In [55]:
import time

drug_func_list= [smiles2morgan,trans_drug,drug2emb_encoder,calcPubChemFingerAll,smiles2mpnnfeature, smiles2daylight]
column_name = 'SMILES'
start = time.time()

for func in drug_func_list:
    save_column_name = func.__name__
    unique = pd.Series(df_data[column_name].unique()).apply(func)
    unique_dict = dict(zip(df_data[column_name].unique(), unique))
    df_data[save_column_name] = [unique_dict[i] for i in df_data[column_name]]
    end = time.time()
    print(end - start)

rdkit not found this smiles for morgan: Cc1n[nH]n(C)c1-c1cnc2c(c1)n([C@H](C1CCOCC1)c1ccccc1)c1cc(ccc21)C(C)(C)O convert to all 0 features
rdkit not found this smiles for morgan: COc1ccc2C[C@@H]3[C@]45CC[C@](OC)([C@@H]6Oc1c2[C@]46CC[N]3(C)CC1CC1)[C@@H](COCc1cc2ccccc2o1)C5 convert to all 0 features
rdkit not found this smiles for morgan: COc1ccc(cc1OC)-c1cc(C(=O)NCc2c(C)cc(C)nc2=O)c2cnn(C(C)C)c2n1 convert to all 0 features
7.282987356185913
7.7704432010650635
8.568466424942017
92.0403687953949
Molecules not found and change to zero vectors..
Molecules not found and change to zero vectors..
Molecules not found and change to zero vectors..
147.95095109939575
rdkit not found this smiles: Cc1n[nH]n(C)c1-c1cnc2c(c1)n([C@H](C1CCOCC1)c1ccccc1)c1cc(ccc21)C(C)(C)O convert to all 0 features
rdkit not found this smiles: COc1ccc2C[C@@H]3[C@]45CC[C@](OC)([C@@H]6Oc1c2[C@]46CC[N]3(C)CC1CC1)[C@@H](COCc1cc2ccccc2o1)C5 convert to all 0 features
rdkit not found this smiles: [I-] convert to all 0 features
r

In [56]:
prot_func_list = [CalculateConjointTriad, trans_protein, protein2emb_encoder,target2quasi]
column_name = 'Target Sequence'
start = time.time()

for func in prot_func_list:
    save_column_name = func.__name__
    AA = pd.Series(df_data[column_name].unique()).apply(func)
    AA_dict = dict(zip(df_data[column_name].unique(), AA))
    df_data[save_column_name] = [AA_dict[i] for i in df_data[column_name]]
    end = time.time()
    print(end - start)

2.031992197036743
2.7403295040130615
3.2397186756134033
Quasi-seq fingerprint not working for smiles: pqitlwkrplvtikiggqlkealldtgaddtvieemslpgrwkpkmiggiggfikvrqydqiiieiaghkaigtvlvgptpvniigrnlltqigatlnf convert to 0 vectors
Quasi-seq fingerprint not working for smiles: MKAFLGALEFQENEYEELKELYESLKTKQKPHTLFISCVDSRVVPNLITGTKPGELYVIRNMGNVIPPKTSHKESLSTMASIEYAIVHVGVQNLIICGHSDCGACGSTHLINDGXTKAKTPYIADWIQFLEPIKEELKNHPQFSNHFAKRSWLTERLNVRLQLNNLLSYDFIQERVVNNELKIFGWHYIIETGRIYNYNFESHFFEPIXETXKQRKSHENF convert to 0 vectors
Quasi-seq fingerprint not working for smiles: MSKSKVDNQFYSVEVGDSTFTVLKRYQNLKPIGSGAQGIVCAAYDAVLDRNVAIKKLSRPFQNQTHAKRAYRELVLMKCVNHKNIISLLNVFTPQKTLEEFQDVYLVMELMDANLCQVIQMELDHERMSYLLYQMLCGIKHLHSAGIIHRDLKPSNIVVKSDCTLKILDFGLARTAGTSFMMTPYVVTRYYRAPEVILGMGYKENVDIWSVGCIMGEMVRHKILFPGRDYIDQWNKVIEQLGTPCPEFMKKLQPTVRNYVENRPKYAGLTFPKLFPDSLFPADSEHNKLKASQARDLLSKMLVIDPAKRISVDDALQHPYINVWYDPAXXXXXDEREHTIEEWKELIYKEVMNSE convert to 0 vectors
Quasi-seq fingerprint not working for smiles: pqitlwkrplvtikiggql

In [57]:
import sys
np.set_printoptions(threshold = sys.maxsize)

df_data.to_pickle("./df_data_small3.pkl")
df_data.to_csv('df_data_small3.csv.gz'
         , sep='|'
         , header=True
         , index=False
         , chunksize=1000
         , compression='gzip'
         , encoding='utf-8')

In [152]:
df_data_out = pd.read_csv("./df_data_small4.csv.gz"
         , sep='|'
         , compression='gzip'
         , encoding='utf-8')
                         #dtype =  {'smiles2morgan': np.ndarray, 'drug2emb_encoder': np.ndarray, 'CalculateConjointTriad': np.ndarray, 'protein2emb_encoder': np.ndarray} )

FileNotFoundError: [Errno 2] No such file or directory: './df_data_small4.csv.gz'

In [151]:
df_data_out['trans_drug'].iloc[1]

"['F', 'C', '(', 'F', ')', '(', 'F', ')', 'c', '1', 'c', 'c', 'c', '2', 'N', '=', 'C', '(', 'C', 'C', '(', '=', 'O', ')', 'N', 'c', '2', 'c', '1', ')', 'c', '1', 'c', 'c', 'c', 'c', '(', 'c', '1', ')', '-', 'n', '1', 'c', 'c', 'n', 'c', '1', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?', '?']"

In [87]:
#turn categorical variables into numerical dummy variables for modeling

cat_list = pd.get_dummies(df_data['Organism'], prefix='var')
df_data1=df_data.join(cat_list)

In [88]:
df_data1.head()

,Kd,IC50,Ki,EC50,Temp,pH,SMILES,Target Sequence,Organism,Kd_raw,...,var_Trypanosoma brucei brucei,var_Trypanosoma brucei rhodesiense,var_Trypanosoma cruzi,var_Vibrio fischeri,var_Vibrio harveyi,var_Woolly monkey sarcoma virus,var_Xenopus sp.,var_Yersinia enterocolitica,var_Zea mays,var_sequence
177623,872669.450093,1.264663e+08,1.000000e+04,362878.629538,27.936324,7.242816,NC(CCS(O)(=O)=O)C(O)=O,MLLWVQQALLALLLPTLLAQGEARRSRNTTRPALLRLSDYLLTNYR...,Homo sapiens,NaN,...,0,0,0,0,0,0,0,0,0,0
1172701,872669.450093,2.600000e+04,1.941028e+06,362878.629538,27.936324,7.242816,CCCc1ccc(Br)cc1S(=O)(=O)Nc1onc(C)c1C,MQPPPSLCGRALVALVLACGLSRIWGEERGFPPDRATPLLQTAEIM...,Homo sapiens,NaN,...,0,0,0,0,0,0,0,0,0,0
153956,872669.450093,1.264663e+08,1.413000e+02,362878.629538,27.936324,7.242816,Oc1ccc2C[C@H]3N(CC=C)CC[C@@]45[C@@H](Oc1c24)C(...,MEPAPSAGAELQPPLFANASDAYPSACPSAGANASGPPGARSASSL...,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
850382,872669.450093,1.264663e+08,5.600000e+01,362878.629538,27.936324,7.242816,Cc1cc(O)cc(C)c1C[C@H](N)C(=O)NCCCCc1nc(C)c(CCc...,MEPAPSAGAELQPPLFANASDAYPSACPSAGANASGPPGARSASSL...,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1145273,872669.450093,3.000000e+04,1.941028e+06,362878.629538,27.936324,7.242816,CNc1nc(NC(=O)Nc2ccccc2)n2nc(nc2n1)-c1ccco1,MLLETQDALYVALELVIAALSVAGNVLVCAAVGTANTLQTPTNYFL...,Homo sapiens,NaN,...,0,0,0,0,0,0,0,0,0,0


In [62]:
discard=['SMILES','Target Sequence','Organism','trans_drug','trans_protein','pH_raw','Kd_raw','IC50_raw','Ki_raw','EC50_raw','Temp_raw']
df_vars=df_data1.columns.values.tolist()
to_keep=[i for i in df_vars if i not in discard]
df_final=df_data1[to_keep]
df_final.head()

,Kd,IC50,Ki,EC50,Temp,pH,magic50,smiles2morgan,drug2emb_encoder,calcPubChemFingerAll,...,var_Trypanosoma brucei brucei,var_Trypanosoma brucei rhodesiense,var_Trypanosoma cruzi,var_Vibrio fischeri,var_Vibrio harveyi,var_Woolly monkey sarcoma virus,var_Xenopus sp.,var_Yersinia enterocolitica,var_Zea mays,var_sequence
1264617,872669.450093,2600.0,1.941028e+06,362878.629538,27.936324,7.242816,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([1529, 838, 1701, 973, 517, 482, 587, 403, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
1533686,872669.450093,43.0,1.941028e+06,362878.629538,27.936324,7.242816,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([450, 205, 221, 538, 2068, 290, 37, 35, 168, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0
1125111,872669.450093,9700.0,1.941028e+06,362878.629538,27.936324,7.242816,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([54, 32, 200, 35, 295, 370, 246, 646, 123, 99...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
225474,872669.450093,6.3,1.941028e+06,362878.629538,27.936324,7.400000,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([43, 517, 164, 106, 168, 381, 266, 381, 343, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0
652860,872669.450093,100.0,1.941028e+06,362878.629538,27.936324,7.242816,2,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","([1043, 343, 158, 78, 97, 133, 99, 1074, 139, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0


In [86]:
#X["smiles2morgan"].iloc[1]
df_data.head()

,Kd,IC50,Ki,EC50,Temp,pH,SMILES,Target Sequence,Organism,Kd_raw,...,smiles2morgan,trans_drug,drug2emb_encoder,calcPubChemFingerAll,smiles2mpnnfeature,smiles2daylight,CalculateConjointTriad,trans_protein,protein2emb_encoder,target2quasi
177623,872669.450093,1.264663e+08,1.000000e+04,362878.629538,27.936324,7.242816,NC(CCS(O)(=O)=O)C(O)=O,MLLWVQQALLALLLPTLLAQGEARRSRNTTRPALLRLSDYLLTNYR...,Homo sapiens,NaN,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[N, C, (, C, C, S, (, O, ), (, =, O, ), =, O, ...","([117, 80, 22, 96, 1049, 43, 96, 180, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ...","[[[tensor(0.), tensor(1.), tensor(0.), tensor(...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 5, 4, 0, 3, 0, 2, 4, 8, 4, 1, 1, 3, 0, 5, ...","[M, L, L, W, V, Q, Q, A, L, L, A, L, L, L, P, ...","([780, 23, 1062, 176, 1066, 231, 2689, 180, 48...","[0.000103, 0.000139, 7.1e-05, 0.000107, 3.6e-0..."
1172701,872669.450093,2.600000e+04,1.941028e+06,362878.629538,27.936324,7.242816,CCCc1ccc(Br)cc1S(=O)(=O)Nc1onc(C)c1C,MQPPPSLCGRALVALVLACGLSRIWGEERGFPPDRATPLLQTAEIM...,Homo sapiens,NaN,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[C, C, C, c, 1, c, c, c, (, B, r, ), c, c, 1, ...","([103, 95, 1816, 195, 1144, 574, 445, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ...","[[[tensor(1.), tensor(0.), tensor(0.), tensor(...","[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...","[1, 5, 4, 0, 1, 2, 0, 9, 6, 7, 3, 2, 4, 2, 2, ...","[M, Q, P, P, P, S, L, C, G, R, A, L, V, A, L, ...","([300, 715, 31, 152, 401, 356, 28, 173, 367, 1...","[0.000168, 0.000108, 9.8e-05, 7.7e-05, 0.00010..."
153956,872669.450093,1.264663e+08,1.413000e+02,362878.629538,27.936324,7.242816,Oc1ccc2C[C@H]3N(CC=C)CC[C@@]45[C@@H](Oc1c24)C(...,MEPAPSAGAELQPPLFANASDAYPSACPSAGANASGPPGARSASSL...,NaN,NaN,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[O, c, 1, c, c, c, 2, C, [, C, ?, H, ], 3, N, ...","([54, 856, 269, 261, 85, 284, 526, 6, 459, 785...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...","[[[tensor(0.), tensor(0.), tensor(1.), tensor(...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[9, 6, 1, 1, 1, 1, 4, 12, 7, 6, 1, 0, 1, 0, 6,...","[M, E, P, A, P, S, A, G, A, E, L, Q, P, P, L, ...","([3426, 404, 41, 233, 163, 47, 1470, 763, 114,...","[0.000399, 0.000196, 8.5e-05, 0.00012, 0.00012..."
850382,872669.450093,1.264663e+08,5.600000e+01,362878.629538,27.936324,7.242816,Cc1cc(O)cc(C)c1C[C@H](N)C(=O)NCCCCc1nc(C)c(CCc...,MEPAPSAGAELQPPLFANASDAYPSACPSAGANASGPPGARSASSL...,NaN,NaN,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[C, c, 1, c, c, (, O, ), c, c, (, C, ), c, 1, ...","([457, 225, 72, 445, 664, 2581, 1548, 1685, 26...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...","[[[tensor(1.), tensor(0.), tensor(0.), tensor(...","[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","[9, 6, 1, 1, 1, 1, 4, 12, 7, 6, 1, 0, 1, 0, 6,...","[M, E, P, A, P, S, A, G, A, E, L, Q, P, P, L, ...","([3426, 404, 41, 233, 163, 47, 1470, 763, 114,...","[0.000399, 0.000196, 8.5e-05, 0.00012, 0.00012..."
1145273,872669.450093,3.000000e+04,1.941028e+06,362878.629538,27.936324,7.242816,CNc1nc(NC(=O)Nc2ccccc2)n2nc(nc2n1)-c1ccco1,MLLETQDALYVALELVIAALSVAGNVLVCAAVGTANTLQTPTNYFL...,Homo sapiens,NaN,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[C, N, c, 1, n, c, (, N, C, (, =, O, ), N, c, ...","([2275, 249, 265, 1460, 187, 593, 35, 127, 27,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[[[tensor(1.), tensor(0.), tensor(0.), tensor(...","[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, ...","[7, 8, 1, 4, 1, 2, 1, 7, 6, 7, 1, 0, 1, 3, 3, ...","[M, L, L, E, T, Q, D, A, L, Y, V, A, L, E, L, ...","([780, 263, 156, 1376, 356, 30, 57, 183, 290, ...","[0.000416, 0.000139, 0.000185, 0.000104, 0.000..."


In [75]:
def flattener(x):
    if isinstance(x, collections.Iterable):
        if isinstance(x,torch.Tensor):
            y = x.numpy()
            return [a for i in y for a in flattener(i)]
        else:
            return [a for i in x for a in flattener(i)]
    else:
        return [x] 

In [95]:
X = df_final.drop(["IC50","magic50","smiles2mpnnfeature","Kd","Ki","EC50"], axis=1)

In [96]:
X.head()
#dtype =  {'smiles2morgan': np.ndarray, 'drug2emb_encoder': np.ndarray, 'CalculateConjointTriad': np.ndarray, 'protein2emb_encoder': np.ndarray} 


,Temp,pH,smiles2morgan,drug2emb_encoder,calcPubChemFingerAll,smiles2daylight,CalculateConjointTriad,protein2emb_encoder,target2quasi,var_Agaricus bisporus,...,var_Trypanosoma brucei brucei,var_Trypanosoma brucei rhodesiense,var_Trypanosoma cruzi,var_Vibrio fischeri,var_Vibrio harveyi,var_Woolly monkey sarcoma virus,var_Xenopus sp.,var_Yersinia enterocolitica,var_Zea mays,var_sequence
1264617,27.936324,7.242816,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([1529, 838, 1701, 973, 517, 482, 587, 403, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4, 13, 9, 2, 7, 6, 1, 11, 11, 7, 6, 7, 15, 2,...","([1560, 1452, 100, 3437, 45, 252, 448, 624, 70...","[3.6e-05, 2.4e-05, 2.5e-05, 4.4e-05, 1.3e-05, ...",0,...,0,0,0,0,0,0,0,0,0,0
1533686,27.936324,7.242816,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([450, 205, 221, 538, 2068, 290, 37, 35, 168, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ...","[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...","[10, 15, 11, 6, 6, 5, 1, 14, 14, 9, 3, 2, 8, 3...","([1426, 25, 176, 176, 25, 224, 39, 345, 14, 30...","[0.000127, 8.2e-05, 3.5e-05, 4.4e-05, 4.1e-05,...",0,...,0,0,0,0,0,0,0,0,0,0
1125111,27.936324,7.242816,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([54, 32, 200, 35, 295, 370, 246, 646, 123, 99...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","[4, 5, 3, 3, 2, 2, 0, 1, 3, 5, 3, 4, 1, 0, 3, ...","([186, 44, 133, 109, 1152, 143, 83, 589, 39, 8...","[0.000147, 0.000111, 8e-05, 0.000123, 3.6e-05,...",0,...,0,0,0,0,0,0,0,0,0,0
225474,27.936324,7.400000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","([43, 517, 164, 106, 168, 381, 266, 381, 343, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...","[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...","[4, 5, 3, 1, 1, 1, 1, 1, 7, 1, 2, 3, 5, 0, 2, ...","([874, 240, 125, 284, 43, 67, 58, 556, 1765, 8...","[0.000232, 0.000242, 7.3e-05, 0.000178, 4.1e-0...",0,...,0,0,0,0,0,0,0,0,0,0
652860,27.936324,7.242816,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","([1043, 343, 158, 78, 97, 133, 99, 1074, 139, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...","[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[5, 10, 2, 0, 4, 0, 2, 4, 4, 6, 0, 4, 2, 0, 4,...","([550, 487, 189, 920, 1121, 98, 44, 946, 47, 2...","[6e-05, 7.1e-05, 4.8e-05, 5.6e-05, 2.4e-05, 0....",0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
typelist = []
for i in list(X):
    print(i, ':', type(X[i].iloc[1]))#, ':', X[i].iloc[1])
    if isinstance(X[i].iloc[1],np.ndarray):
        print(len(X[i].iloc[1]))
        typelist.extend([i]*len(X[i].iloc[1]))
    elif isinstance(X[i].iloc[1],tuple):
        for n in X[i].iloc[1]:
            if isinstance(n,np.ndarray):
                print(len(n))
                typelist.extend([i]*len(n))
    elif i[:4] == 'var_':
        typelist.append('organism')
    else:
        typelist.append(i)

Temp : <class 'numpy.float64'>
pH : <class 'numpy.float64'>
smiles2morgan : <class 'numpy.ndarray'>
1024
drug2emb_encoder : <class 'tuple'>
50
50
calcPubChemFingerAll : <class 'numpy.ndarray'>
881
smiles2daylight : <class 'numpy.ndarray'>
2048
CalculateConjointTriad : <class 'numpy.ndarray'>
343
protein2emb_encoder : <class 'tuple'>
545
545
target2quasi : <class 'numpy.ndarray'>
100
var_Agaricus bisporus : <class 'numpy.uint8'>
var_Aquifex aeolicus : <class 'numpy.uint8'>
var_Arabidopsis thaliana : <class 'numpy.uint8'>
var_Bacillus anthracis : <class 'numpy.uint8'>
var_Bacillus lentus : <class 'numpy.uint8'>
var_Bacillus subtilis : <class 'numpy.uint8'>
var_Bacillus thermoproteolyticus : <class 'numpy.uint8'>
var_Bison bison : <class 'numpy.uint8'>
var_Bos taurus : <class 'numpy.uint8'>
var_Brucella suis : <class 'numpy.uint8'>
var_Caenorhabditis elegans : <class 'numpy.uint8'>
var_Caleucoela sp. 2 MB402 : <class 'numpy.uint8'>
var_Canavalia ensiformis : <class 'numpy.uint8'>
var_Cand

In [70]:
X['smiles2mpnnfeature'].iloc[1]
Z = X['smiles2mpnnfeature']

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 1.],
         [0., 1., 0.,  ..., 0., 0., 1.],
         [1., 0., 0.,  ..., 0., 0., 1.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[ 2.,  0.,  0.,  0.,  0.,  0.],
         [ 1.,  4.,  6.,  8.,  0.,  0.],
         [ 3.,  0.,  0.,  0.,  0.,  0.],
         [ 5.,  0.,  0.,  0.,  0.,  0.],
         [ 7., 10., 53.,  0.,  0.,  0.],
         [ 9., 12.,  0.,  0.,  0.,  0.],
         [11., 14.,  0.,  0.,  0.,  0.],
         [13., 16., 59.,  0.,  0.,  0.],
         [15., 18.,  0.,  0.,  0.,  0.],
         [17., 20., 32.,  0.,  0.,  0.],
         [19., 22.,  0.,  0.,  0.,  0.],
         [21., 24., 26., 

In [98]:
vec = X.apply(flattener,axis=1)
foo = vec.to_numpy(dtype = object)
foo2 = np.vstack(foo)

In [ ]:
from sklearn.model_selection import train_test_split
y = df_final["IC50"]
X_train, X_test, y_train, y_test = train_test_split(foo2, y, test_size=0.2)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_scaled = scaler.fit_transform(X_train)
test_scaled = scaler.transform(X_test)

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
tree_model = DecisionTreeRegressor()

In [102]:
tree_model.fit(train_scaled, y_train)
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import average_precision_score
tree_mse = mean_squared_error(y_train, tree_model.predict(train_scaled))
tree_mae = mean_absolute_error(y_train, tree_model.predict(train_scaled))
from math import sqrt
print("Decision Tree training mse = ",tree_mse," & mae = ",tree_mae," & rmse = ", sqrt(tree_mse))
tree_test_mse = mean_squared_error(y_test, tree_model.predict(test_scaled))
tree_test_mae = mean_absolute_error(y_test, tree_model.predict(test_scaled))
print("Decision Tree test mse = ",tree_test_mse," & mae = ",tree_test_mae," & rmse = ", sqrt(tree_test_mse))
a = pearsonr(y_train, tree_model.predict(train_scaled))
b = pearsonr(y_test, tree_model.predict(test_scaled))
print("Decision Tree train r = ",a)
print("Decision Tree test r = ",b)

Decision Tree training mse =  999607184856.336  & mae =  15808.434643801334  & rmse =  999803.5731364116
Decision Tree test mse =  4031680966584584.5  & mae =  31858931.08447345  & rmse =  63495519.263839275
Decision Tree train r =  (0.9998672684848211, 0.0)
Decision Tree test r =  (0.4696651336365103, 2.856528750729771e-110)


In [103]:
importances = pd.DataFrame({'importance':np.round(tree_model.feature_importances_,3)})
out = importances.sort_values('importance',ascending=False)
importances["Type"] = typelist
type_importance = importances.groupby(by=['Type']).sum()
type_importance.sort_values('importance',ascending=False)

,importance
Type,
target2quasi,0.233
smiles2daylight,0.215
smiles2morgan,0.202
protein2emb_encoder,0.181
CalculateConjointTriad,0.076
calcPubChemFingerAll,0.035
drug2emb_encoder,0.028
organism,0.001
pH,0.001


In [121]:
rf_model = RandomForestRegressor(n_estimators=10)
rf_model.fit(train_scaled, y_train)

RandomForestRegressor(n_estimators=10)

In [122]:
rf_mse = mean_squared_error(y_train, rf_model.predict(train_scaled))
rf_mae = mean_absolute_error(y_train, rf_model.predict(train_scaled))

print("Random Forest training mse = ",rf_mse," & mae = ",rf_mae," & rmse = ", sqrt(rf_mse))
rf_test_mse = mean_squared_error(y_test, rf_model.predict(test_scaled))
rf_test_mae = mean_absolute_error(y_test, rf_model.predict(test_scaled))
print("Random Forest test mse = ",rf_test_mse," & mae = ",rf_test_mae," & rmse = ", sqrt(rf_test_mse))
a = pearsonr(y_train, rf_model.predict(train_scaled))
b = pearsonr(y_test, rf_model.predict(test_scaled))
print("Random Forest train r = ",a)
print("Random Forest test r = ",b)

Random Forest training mse =  423815254928228.75  & mae =  13340291.9462728  & rmse =  20586773.78629854
Random Forest test mse =  2464060344209559.0  & mae =  36863545.149093814  & rmse =  49639302.41461456
Random Forest train r =  (0.9518525611786604, 0.0)
Random Forest test r =  (0.5974091923873439, 8.844622493480001e-194)


In [130]:
importances = pd.DataFrame({'importance':np.round(rf_model2.feature_importances_,3)})
out = importances.sort_values('importance',ascending=False)
importances["Type"] = typelist
type_importance = importances.groupby(by=['Type']).sum()
type_importance.sort_values('importance',ascending=False)

,importance
Type,
protein2emb_encoder,0.214
target2quasi,0.150
smiles2daylight,0.105
CalculateConjointTriad,0.082
smiles2morgan,0.051
drug2emb_encoder,0.036
calcPubChemFingerAll,0.011
Temp,0.004
pH,0.004


target2quasi	0.208

protein2emb_encoder	0.196

smiles2morgan	0.067

CalculateConjointTriad	0.060

drug2emb_encoder	0.037

smiles2daylight	0.024

pH	0.003

calcPubChemFingerAll	0.002

Temp	0.001

organism	0.001


target2quasi	0.208
protein2emb_encoder	0.193
smiles2morgan	0.088
smiles2daylight	0.079
CalculateConjointTriad	0.069
drug2emb_encoder	0.041
calcPubChemFingerAll	0.006
pH	0.003
Temp	0.001
organism	0.000

protein2emb_encoder	0.212
target2quasi	0.198
smiles2morgan	0.093
smiles2daylight	0.064
CalculateConjointTriad	0.060
drug2emb_encoder	0.035
calcPubChemFingerAll	0.009
pH	0.003
organism	0.002
Temp	0.001

protein2emb_encoder	0.214
target2quasi	0.150
smiles2daylight	0.105
CalculateConjointTriad	0.082
smiles2morgan	0.051
drug2emb_encoder	0.036
calcPubChemFingerAll	0.011
Temp	0.004
pH	0.004
organism	0.001

In [111]:
y2 = df_final["magic50"]
X2_train, X2_test, y2_train, y2_test = train_test_split(foo2, y2, test_size=0.2)
train2_scaled = scaler.fit_transform(X2_train)
test2_scaled = scaler.transform(X2_test)

In [125]:
tree_model_mag = DecisionTreeRegressor()
tree_model_mag.fit(train2_scaled, y2_train)

DecisionTreeRegressor()

In [126]:
a = pearsonr(y2_train, tree_model_mag.predict(train2_scaled))
b = pearsonr(y2_test, tree_model_mag.predict(test2_scaled))
print("Decision Tree train r = ",a)
print("Decision Tree test r = ",b)

Decision Tree train r =  (0.9998263059892019, 0.0)
Decision Tree test r =  (0.42169792391724403, 4.706595176419894e-87)


In [127]:
rf_model2 = RandomForestRegressor(n_estimators=10)
rf_model2.fit(train2_scaled, y2_train)

RandomForestRegressor(n_estimators=10)

In [128]:
a = pearsonr(y2_train, rf_model2.predict(train2_scaled))
b = pearsonr(y2_test, rf_model2.predict(test2_scaled))
print("Random Forest train r = ",a)
print("Random Forest test r = ",b)

Random Forest train r =  (0.9492116114851832, 0.0)
Random Forest test r =  (0.5703802084905281, 5.688247814058485e-173)


In [114]:
from sklearn.tree import DecisionTreeClassifier
tree_cat = DecisionTreeClassifier()
tree_cat.fit(train2_scaled, y2_train)

DecisionTreeClassifier()

In [115]:
from sklearn.metrics import roc_auc_score
a = roc_auc_score(y2_train, tree_cat.predict(train2_scaled))
b = roc_auc_score(y2_test, tree_cat.predict(test2_scaled))
print("Decision Tree train r = ",a)
print("Decision Tree test r = ",b)

ValueError: multi_class must be in ('ovo', 'ovr')

In [132]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline

# Always scale the input. The most convenient way is to use a pipeline.
reg = make_pipeline(StandardScaler(), SGDRegressor(max_iter=1000, tol=1e-3))
reg.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdregressor', SGDRegressor())])

In [133]:
reg.score(X_test,y_test)

-494628951.62132967

In [134]:
a = pearsonr(y_train, reg.predict(train_scaled))
b = pearsonr(y_test, reg.predict(test_scaled))
print("Random Forest train r = ",a)
print("Random Forest test r = ",b)

Random Forest train r =  (-0.08219424044243488, 1.7983828615846871e-13)
Random Forest test r =  (-0.03611986779394279, 0.10634390878949515)


In [154]:
from DeepPurpose import DTI as models
#run this if you want to use my trained one
model_9_24 = models.model_pretrained(path_dir = './model-9-24')

In [155]:
model

In [143]:
state_dict = torch.load(r'C:\Users\Julia\Dropbox\Work\insight\omic\DeepPurpose-omic\model-9-24\model.pt')
#state_dict = torch.load('./model-9-24', map_location = torch.device('cpu'))

In [144]:
state_dict

OrderedDict([('model_drug.predictor.0.weight',
              tensor([[ 0.0867,  0.1223,  0.1496,  ..., -0.2063,  0.0726, -0.0682],
                      [-0.1626, -0.1736, -0.2029,  ..., -0.1643,  0.0638, -0.2519],
                      [-0.0012,  0.0083, -0.1177,  ..., -0.0029, -0.1655,  0.0157],
                      ...,
                      [ 0.1376,  0.0517, -0.1924,  ...,  0.1951, -0.0767,  0.0975],
                      [-0.1610,  0.0434,  0.0504,  ...,  0.1538, -0.2475, -0.0231],
                      [-0.1537, -0.0240,  0.0892,  ...,  0.2453,  0.0534, -0.2827]])),
             ('model_drug.predictor.0.bias',
              tensor([-0.1856, -0.1796, -0.0606,  ..., -0.1561, -0.2625, -0.2159])),
             ('model_drug.predictor.1.weight',
              tensor([[-0.0469, -0.0254, -0.0642,  ..., -0.0838,  0.0239, -0.0942],
                      [-0.0219,  0.0886, -0.0970,  ..., -0.0333, -0.0350,  0.0058],
                      [ 0.1033, -0.0467, -0.1647,  ..., -0.0916, -0.0458, 

In [ ]:
if self.device == 'cuda':
			state_dict = torch.load(path)
		else:
			state_dict = torch.load(path, map_location = torch.device('cpu'))
		# to support training from multi-gpus data-parallel:
        
		if next(iter(state_dict))[:7] == 'module.':
			# the pretrained model is from data-parallel module
			from collections import OrderedDict
			new_state_dict = OrderedDict()
			for k, v in state_dict.items():
				name = k[7:] # remove `module.`
				new_state_dict[name] = v
			state_dict = new_state_dict

		self.model.load_state_dict(state_dict)

		self.binary = self.config['binary']


In [158]:
#model1 = model_9_24.model.load_state_dict(state_dict)
model1 = model_9_24.model

In [159]:
type(model1)

DeepPurpose.DTI.Classifier

In [161]:
new_classifier = nn.Sequential(*list(model1.children())[:-1])

In [162]:
type(new_classifier)

torch.nn.modules.container.Sequential

In [166]:
drug_encoding, target_encoding = 'MPNN', 'Conjoint_triad'
X_drug = ['CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC=C4)N']
X_target = ['MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAPVRQQPKVQTTPPPAVQGQKVGSLTPPSSPKTQRAGHRRILSDVTHSAVFGVPASKSTQLLQAAAAEASLNKSKSATTTPSGSPRTSQQNVYNPSEGSTWNPFDDDNFSKLTAEELLNKDFAKLGEGKHPEKLGGSAESLIPGFQSTQGDAFATTSFSAGTAEKRKGGQTVDSGLPLLSVSDPFIPLQVPDAPEKLIEGLKSPDTSLLLPDLLPMTDPFGSTSDAVIEKADVAVESLIPGLEPPVPQRLPSQTESVTSNRTDSLTGEDSLLDCSLLSNPTTDLLEEFAPTAISAPVHKAAEDSNLISGFDVPEGSDKVAEDEFDPIPVLITKNPQGGHSRNSSGSSESSLPNLARSLLLVDQLIDL']
X_pred = data_process(X_drug, X_target, y_test, 
                                drug_encoding, target_encoding, 
                                split_method='no_split')
y_pred = new_classifier.predict(X_pred)

Drug Target Interaction Prediction Mode...
in total: 1 drug-target pairs
encoding drug...
unique drugs: 1
encoding protein...
unique target sequence: 1
splitting dataset...
do not do train/test split on the data for already splitted data


AttributeError: 'Sequential' object has no attribute 'predict'

In [167]:
new_classifier.eval()

Sequential(
  (0): MLP(
    (predictor): ModuleList(
      (0): Linear(in_features=1024, out_features=1024, bias=True)
      (1): Linear(in_features=1024, out_features=256, bias=True)
      (2): Linear(in_features=256, out_features=64, bias=True)
      (3): Linear(in_features=64, out_features=128, bias=True)
    )
  )
  (1): MLP(
    (predictor): ModuleList(
      (0): Linear(in_features=343, out_features=1024, bias=True)
      (1): Linear(in_features=1024, out_features=256, bias=True)
      (2): Linear(in_features=256, out_features=64, bias=True)
      (3): Linear(in_features=64, out_features=256, bias=True)
    )
  )
  (2): Dropout(p=0.1, inplace=False)
)

In [168]:
model1.eval()

Classifier(
  (model_drug): MLP(
    (predictor): ModuleList(
      (0): Linear(in_features=1024, out_features=1024, bias=True)
      (1): Linear(in_features=1024, out_features=256, bias=True)
      (2): Linear(in_features=256, out_features=64, bias=True)
      (3): Linear(in_features=64, out_features=128, bias=True)
    )
  )
  (model_protein): MLP(
    (predictor): ModuleList(
      (0): Linear(in_features=343, out_features=1024, bias=True)
      (1): Linear(in_features=1024, out_features=256, bias=True)
      (2): Linear(in_features=256, out_features=64, bias=True)
      (3): Linear(in_features=64, out_features=256, bias=True)
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (predictor): ModuleList(
    (0): Linear(in_features=384, out_features=1024, bias=True)
    (1): Linear(in_features=1024, out_features=1024, bias=True)
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=1, bias=True)
  )
)